In [8]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from collections import Counter

# Imports de la libreria propia
from vecopsciml.kernels.derivative import DerivativeKernels
from vecopsciml.utils import TensOps

# Imports de las funciones creadas para este programa
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

Using device: cuda


In [10]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'/home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning'
DATA_PATH = os.path.join(ROOT_PATH, r'data/non_linear/non_linear_decomposition.pkl')
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results/non_linear')
MODEL_RESULTS_PATH = os.path.join(ROOT_PATH, r'results/non_linear/model_autoencoder')

# Creamos las carpetas que sean necesarias (si ya están creadas se avisará de ello)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_PATH)

Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear
Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear/model_autoencoder


In [11]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/data/non_linear/non_linear_decomposition.pkl


In [12]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

## División de los datos

In [64]:
X_train = torch.Tensor(dataset['X_train']).unsqueeze(1)
y_train = torch.Tensor(dataset['y_train']).unsqueeze(1)
K_train = torch.tensor(dataset['k_train']).unsqueeze(1)
f_train = torch.tensor(dataset['f_train']).unsqueeze(1)

X_val = torch.Tensor(dataset['X_val']).unsqueeze(1)
y_val = TensOps(torch.Tensor(dataset['y_val']).unsqueeze(1).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val']).unsqueeze(1).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val']).unsqueeze(1).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

print("Train dataset length:", len(X_train))
print("Validation dataset length:", len(X_val))

Train dataset length: 8000
Validation dataset length: 2000


In [65]:
N_data_AE = len(X_train)//4
N_data_NN = len(X_train) - len(X_train)//4
prop_data_NN = 1 - N_data_AE/(N_data_NN + N_data_AE)

print("Dataset length for the autoencoder:", N_data_AE)
print("Dataset length for the PGNNIV:", N_data_NN)

X_AE, X_NN, y_AE, y_NN, K_AE, K_NN, f_AE, f_NN = train_test_split(X_train, y_train, K_train, f_train, test_size=prop_data_NN, random_state=42)

Dataset length for the autoencoder: 2000
Dataset length for the PGNNIV: 6000


#### Datos para el autoencoder

In [70]:
y_train_AE, y_test_AE = train_test_split(y_AE, test_size=0.2, random_state=42)

y_train = TensOps(y_train_AE.requires_grad_(True).to(device), space_dimension=2, contravariance=0, covariance=0)
y_test = TensOps(y_test_AE.requires_grad_(True).to(device), space_dimension=2, contravariance=0, covariance=0)

#### Datos para la PGNNIV

In [73]:
X_train_NN, X_test_NN, y_train_NN, y_test_NN, K_train_NN, K_test_NN, f_train_NN, f_test_NN = train_test_split(X_NN, y_NN, K_NN, f_NN, test_size=0.2, random_state=42)

X_train = X_train_NN.to(device)
X_test = X_test_NN.to(device)

y_train = TensOps(y_train_NN.requires_grad_(True).to(device), space_dimension=2, contravariance=0, covariance=0)
y_test = TensOps(y_test_NN.requires_grad_(True).to(device), space_dimension=2, contravariance=0, covariance=0)

K_train = TensOps(K_train_NN.to(device), space_dimension=2, contravariance=0, covariance=0)
K_test = TensOps(K_test_NN.to(device), space_dimension=2, contravariance=0, covariance=0)

f_train = TensOps(f_train_NN.to(device), space_dimension=2, contravariance=0, covariance=0)
f_test = TensOps(f_test_NN.to(device), space_dimension=2, contravariance=0, covariance=0)

## Autoencoder

In [78]:
from models import Autoencoder
from trainers.eval import loss_function_autoencoder
from utils.checkpoints import load_checkpoint, save_checkpoint

In [79]:
autoencoder_shape = y_train.values[0].shape

In [80]:
def train_autoencoder_epoch(model, optimizer, X_train, y_train):

    model.train()

    y_pred = model(X_train)

    loss = loss_function_autoencoder(y_train, y_pred)

    optimizer.zero_grad() 
    loss.backward(retain_graph=True)
    optimizer.step()

    return loss


def test_autoencoder_epoch(model, X_test, y_test):

    y_pred = model(X_test)
    loss = loss_function_autoencoder(y_test, y_pred)

    return loss


In [128]:
def autoencoder_train_loop(model, optimizer, y_train, y_test, start_epoch, n_epochs, batch_size, model_results_path, device, lr_updated=None):

    print("Start training")

    if start_epoch > 0:
        
        print(f'Starting from a checkpoint. Epoch {start_epoch}.')
        resume_epoch = start_epoch
        model, optimizer, lists = load_checkpoint(model, optimizer, resume_epoch, model_results_path)

        if lr_updated != None:
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr_updated

        train_total_loss_list = lists['train_total_loss_list']
        train_e_loss_list = lists['train_e_loss_list']
        train_pi1_loss_list = lists['train_pi1_loss_list']
        train_pi2_loss_list = lists['train_pi2_loss_list']
        train_pi3_loss_list = lists['train_pi3_loss_list']

        test_total_loss_list = lists['test_total_loss_list']
        test_e_loss_list = lists['test_e_loss_list']
        test_pi1_loss_list = lists['test_pi1_loss_list']
        test_pi2_loss_list = lists['test_pi2_loss_list']
        test_pi3_loss_list = lists['test_pi3_loss_list']

    else:

        train_total_loss_list = []
        train_total_MSE_list = []
        train_e_loss_list = []
        train_pi1_loss_list = []
        train_pi2_loss_list = []
        train_pi3_loss_list = []

        test_total_loss_list = []
        test_total_MSE_list = []
        test_e_loss_list = []
        test_pi1_loss_list = []
        test_pi2_loss_list = []
        test_pi3_loss_list = []
    
    X_train = y_train.values
    X_test = y_test.values
    
    N_train = X_train.shape[0]
    N_test = X_test.shape[0]

    for epoch_i in range(start_epoch, n_epochs):
        for batch_start in range(0, N_train, batch_size):

            X_batch = X_train[batch_start:(batch_start+batch_size)].to(device)
            y_batch = TensOps(y_train.values[batch_start:(batch_start+batch_size)].to(device), space_dimension=y_train.space_dim, contravariance=y_train.order[0], covariance=y_train.order[1])

        loss = train_autoencoder_epoch(model, optimizer, X_batch, y_batch)

        train_total_loss_list.append((loss.item()/batch_size))
        
        loss_test= test_autoencoder_epoch(model, X_test, y_test)

        test_total_loss_list.append((loss_test.item()/N_test))

        if epoch_i % (1 if n_epochs < 100 else (10 if n_epochs <= 1000 else 100)) == 0:
            print(f'Epoch {epoch_i}, Train loss: {loss.item()/batch_size:.3e}, Test loss: {loss_test.item()/N_test:.3e}')

        if epoch_i % (int(n_epochs/10)) == 0:
            save_checkpoint(model, optimizer, epoch_i, model_results_path, 
                            train_total_loss_list=train_total_loss_list, train_e_loss_list=train_e_loss_list, 
                            train_pi1_loss_list=train_pi1_loss_list, train_pi2_loss_list=train_pi2_loss_list, train_pi3_loss_list=train_pi3_loss_list,
                            test_total_loss_list=test_total_loss_list, test_e_loss_list=test_e_loss_list,
                            test_pi1_loss_list=test_pi1_loss_list, test_pi2_loss_list=test_pi2_loss_list, test_pi3_loss_list=test_pi3_loss_list
                            )

    print("\nProceso finalizado después de", n_epochs, "épocas\n")

    

In [129]:
model = Autoencoder(input_size=u_shape, encoding_dim=20, output_size=u_shape, device=device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

autoencoder_train_loop(model, optimizer, y_train, y_test, 0, 50000, 64, MODEL_RESULTS_PATH, device, lr_updated=None)

Start training
Epoch 0, Train loss: 1.056e+02, Test loss: 1.266e+02
Epoch 100, Train loss: 6.187e+01, Test loss: 7.721e+01
Epoch 200, Train loss: 3.295e+01, Test loss: 4.318e+01
Epoch 300, Train loss: 1.572e+01, Test loss: 2.166e+01
Epoch 400, Train loss: 7.636e+00, Test loss: 1.057e+01
Epoch 500, Train loss: 4.708e+00, Test loss: 6.028e+00
Epoch 600, Train loss: 3.789e+00, Test loss: 4.396e+00
Epoch 700, Train loss: 3.516e+00, Test loss: 3.832e+00
Epoch 800, Train loss: 3.434e+00, Test loss: 3.633e+00
Epoch 900, Train loss: 3.403e+00, Test loss: 3.556e+00
Epoch 1000, Train loss: 3.386e+00, Test loss: 3.519e+00
Epoch 1100, Train loss: 3.372e+00, Test loss: 3.493e+00
Epoch 1200, Train loss: 3.360e+00, Test loss: 3.471e+00
Epoch 1300, Train loss: 3.346e+00, Test loss: 3.450e+00
Epoch 1400, Train loss: 3.330e+00, Test loss: 3.429e+00
Epoch 1500, Train loss: 3.311e+00, Test loss: 3.405e+00
Epoch 1600, Train loss: 3.287e+00, Test loss: 3.377e+00
Epoch 1700, Train loss: 3.257e+00, Test loss:

In [130]:
encoder = model.encoder
decoder = model.decoder

## Red neuronal general